# Сultural heritage in Moscow
Мой знакомый спросил меня - как по твоему, какие районы города наиболее интересные с точки зрения объектов культурного наследия и при этом имеющин не самые высокие цены на квартиры?
В данном проекте мы попытаемся ответить на этот вопрос: оценить и классифицировать объекты культурного наследия (далее ОКН) в г.Москве, Россия. Определим наиболее интересные районы города с точки зрения оценок пользователей сервиса Foursquare и соотнесем эти данные со средней стоимостью квартир в том или  инном районе.
### Современная ситуация в Москве
На территории города Москвы расположено более 7 500 объектов, находящихся под государственной охраной.
Наибольшее число Объектов сконцентрировано в Центральном административном округе и самые высокие цены на недвижемость.
### А знаете ли вы, что?
На территории города Москвы расположено 3 объекта культурного наследия, включенных в Список всемирного культурного и природного наследия ЮНЕСКО:
* Ансамбль Кремля и Красная площадь
* Церковь Вознесения в Коломенском
* Ансамбль Новодевичьего монастыря

### Набор данных по ОКН находится на портале открытых данных правительства Москвы
Набор данных "Объекты культурного наследия города Москвы" содержит информацию об объектах, находящихся под государственной охраной, а именно: объектах культурного наследия, включенных в единый государственный реестр объектов культурного наследия (памятников истории и культуры) народов Российской Федерации и выявленных объектах культурного наследия.
#### Описание
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/description?versionNumber=6&releaseNumber=19
#### Паcпорт
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/passport?versionNumber=6&releaseNumber=19
#### Вход для разработчиков для получения ключа и чтения документации
https://apidata.mos.ru/
#### Структура данных
https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json

### Данные по стоимости недвижимости в районах

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
# !conda update -n base -c defaults conda
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',500)
import json
import requests #
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
import lxml
import time
print('Done!')

Solving environment: ...working... done

# All requested packages already installed.

Done!


In [238]:
url='https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json'
results = requests.get(url).json()

In [239]:
print(results)
# df_desc = json_normalize(results)

{'Id': 530, 'IdentificationNumber': '7705021556-Obektykulturnogonaslediya', 'CategoryId': 5, 'CategoryCaption': 'Культура', 'DepartmentId': 7, 'DepartmentCaption': 'Департамент культурного наследия города Москвы', 'Caption': 'Объекты культурного наследия', 'Description': 'Объекты культурного наследия и выявленные объекты культурного наследия, расположенные в пределах установленных границ города Москвы.', 'FullDescription': '<p class="MsoNormal"><span style="font-size:9.0pt;\nfont-family:&quot;Tahoma&quot;,&quot;sans-serif&quot;">Набор данных "Объекты культурного\nнаследия города Москвы" содержит информацию об объектах, находящихся под\nгосударственной охраной, а именно: объектах культурного наследия, включенных в\nединый государственный реестр объектов культурного наследия (памятников истории\nи культуры) народов Российской Федерации и выявленных объектах культурного\nнаследия.<o:p></o:p></span></p><p class="MsoNormal"><span style="font-size:9.0pt;\nfont-family:&quot;Tahoma&quot;,&quot

In [277]:
url ='https://apidata.mos.ru/version'
results = requests.get(url)
if results.ok:
    print('Version API is :{}'.format(results.json()['Version']))
else:
    print('"Houston, we have a problem!"----->', results)

Version API is :1


In [280]:
url='https://apidata.mos.ru/v1/datasets/530/rows?api_key=21d82d2344cd3f6e9deba56e6956cffa'
results = requests.get(url)
if results.ok:
    print('It\'s all right!')
else:
    print('"Houston, we have a problem!"----->', results)

It's all right!


In [286]:
results2=results.json()

In [482]:
df=json_normalize(results2, max_level=4)

In [567]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8317 entries, 0 to 8316
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   global_id                  8317 non-null   int64 
 1   Number                     8317 non-null   int64 
 2   Cells.global_id            8317 non-null   int64 
 3   Cells.AISID                8317 non-null   object
 4   Cells.ObjectNameOnDoc      8317 non-null   object
 5   Cells.ObjectName           8317 non-null   object
 6   Cells.EnsembleNameOnDoc    8317 non-null   object
 7   Cells.EnsembleName         8317 non-null   object
 8   Cells.AdmArea              8317 non-null   object
 9   Cells.District             8317 non-null   object
 10  Cells.SecurityStatus       8317 non-null   object
 11  Cells.Location             8317 non-null   object
 12  Cells.Category             8317 non-null   object
 13  Cells.ObjectType           8317 non-null   object
 14  Cells.ge

In [604]:
df['Cells.Location'][9]

[{'Address': 'город Москва, Кутузовский проспект, дом 27',
  'UNOM': 12424,
  'KAD_CC': '77:07:0007002:5448'}]

In [544]:
a={'Length':0, 'Name':'0'}
for index,i in enumerate(df['Cells.Location']):
    if len(i)>a['Length']:
        a={'Length':len(i), 'Name':df['Cells.ObjectNameOnDoc'][index], 'index':index}

print(a)

{'Length': 72, 'Name': 'Достопримечательное место «Хитровка» с окружающей застройкой в составе кварталов 123, 124, 125, 126, 127', 'index': 2035}


In [524]:
df.to_csv('Data.csv')

In [622]:
CLIENT_ID = 'HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB' # your Foursquare ID
CLIENT_SECRET = 'A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 5
radius=30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB
CLIENT_SECRET:A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB


In [623]:
address = 'город Москва, Кутузовский проспект, 27'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

55.7417149 37.5314882


In [624]:
url_4sq ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude,  radius, LIMIT)
print(url_4sq)

https://api.foursquare.com/v2/venues/explore?client_id=HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB&client_secret=A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB&v=20180604&ll=55.7417149,37.5314882&radius=30&limit=5


In [625]:
results_4sq = requests.get(url_4sq).json()
results_4sq['response']['totalResults']

1

In [626]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [627]:
venues = results_4sq['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,id
0,Банзай,Sushi Restaurant,55.74146,37.53148,5a72efbadd70c575a6690d42


In [613]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '58b853ff9435a90a746443be',
   'name': 'Спортзал в Sberbank Agile Home',
   'location': {'lat': 55.74155328079935,
    'lng': 37.53104548803705,
    'labeledLatLngs': [{'label': 'display',
      'lat': 55.74155328079935,
      'lng': 37.53104548803705}],
    'distance': 33,
    'postalCode': '121170',
    'cc': 'RU',
    'city': 'Москва',
    'state': 'Москва',
    'country': 'Россия',
    'formattedAddress': ['121170, Москва', 'Россия']},
   'categories': [{'id': '4bf58dd8d48988d175941735',
     'name': 'Gym / Fitness Center',
     'pluralName': 'Gyms or Fitness Centers',
     'shortName': 'Gym / Fitness',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-58b853ff9435a90a746443be-0'